In [4]:
import os
import torch
import torch.optim as optim
import time
import numpy as np

from network_cifar import *
from tools_cifar import *

torch.multiprocessing.set_sharing_strategy('file_system')


# Paper: Simultaneous Feature Learning and Hash Coding with Deep Neural Networks
# https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Lai_Simultaneous_Feature_Learning_2015_CVPR_paper.pdf
# 1) a sub-network with multiple convolution-pooling layers to capture a representation of images; 
# 2) a divide-and-encode module designed to generate bitwise hash codes; 
# 3) a triplet ranking loss layer for learning good similarity measures. 

class SubNetwork(nn.Module):
    def __init__(self):
        super(SubNetwork, self).__init__()
        # Define the convolution-pooling layers
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.pool = nn.AvgPool2d(2, 2)

    def forward(self, x):
        # Forward pass through the convolution-pooling layers
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        # Reshape the features for the hashing module
        x = x.view(x.size(0), -1)
        return x


class DivideAndEncode(nn.Module):
    def __init__(self, input_dim, hash_bits, epsilon=0.01, beta=10):
        super(DivideAndEncode, self).__init__()
        self.input_dim = input_dim
        self.hash_bits = hash_bits
        self.epsilon = epsilon
        self.beta = beta
        # Define fully connected layers
        self.fc = nn.Linear(input_dim, hash_bits)
        # Define sigmoid activation
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Forward pass through fully connected layers
        x = self.fc(x)
        # Apply sigmoid activation
        x = self.sigmoid(x * self.beta)
        # Apply piece-wise threshold function
        x = self.piecewise_threshold(x)
        return x

    def piecewise_threshold(self, x):
        # Piece-wise threshold function
        thresholded_x = torch.zeros_like(x)
        thresholded_x[x >= 0.5 + self.epsilon] = 1
        thresholded_x[x <= 0.5 - self.epsilon] = 0
        thresholded_x[(x > 0.5 - self.epsilon) & (x < 0.5 + self.epsilon)] = x[(x > 0.5 - self.epsilon) & (x < 0.5 + self.epsilon)]
        return thresholded_x

class SFHC(nn.Module):
    def __init__(self, hash_bits):
        super(SFHC, self).__init__()
        # Create shared sub-network and divide-and-encode modules
        self.shared_subnetwork = SubNetwork()
        self.hashing_module = DivideAndEncode(128 * 8 * 8, hash_bits)

    def forward(self, x_anchor, x_positive, x_negative):
        # Forward pass through the shared sub-network
        x_anchor = self.shared_subnetwork(x_anchor)
        x_positive = self.shared_subnetwork(x_positive)
        x_negative = self.shared_subnetwork(x_negative)
        # Forward pass through the divide-and-encode module
        hash_code_anchor = self.hashing_module(x_anchor)
        hash_code_positive = self.hashing_module(x_positive)
        hash_code_negative = self.hashing_module(x_negative)
        return hash_code_anchor, hash_code_positive, hash_code_negative
    

def get_config():
    config = {
        "alpha": 0.1,
        # "optimizer":{"type":  optim.SGD, "optim_params": {"lr": 0.05, "weight_decay": 10 ** -5}},
        "optimizer": {"type": optim.RMSprop, "optim_params": {"lr": 1e-5, "weight_decay": 10 ** -5}},
        "info": "[SFHC]",
        "resize_size": 256,
        "crop_size": 224,
        "batch_size": 64,
        "net": SFHC,
        # "net":ResNet,
        "dataset": "cifar10",
        # "dataset": "cifar10-1",
        # "dataset": "pathmnist",
        "epoch": 250,
        "test_map": 15,
        "save_path": "save/SFHC",
        # "device":torch.device("cpu"),
        "device": torch.device("cuda:1"),
        "bit_list": [48],
    }
    config = config_dataset(config)
    return config


def train_SFHC(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        # Forward pass
        output_anchor, output_positive, output_negative = model(data, data, data)
        # Compute the loss
        loss = criterion(output_anchor, output_positive, output_negative)
        # Backward pass
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

class SFHCLoss(torch.nn.Module):
    def __init__(self, config, bit):
        super(SFHCLoss, self).__init__()

    def forward(self, b, b_pos, b_neg):
        # compute the L2 norms of the diffs
        diff_pos = torch.norm(b - b_pos, p=2, dim=1) ** 2
        diff_neg = torch.norm(b - b_neg, p=2, dim=1) ** 2

        # compute the triplet ranking hinge loss
        loss = torch.max(0, diff_pos - diff_neg + 1)

        return loss.mean()


def train_val(config, bit):
    device = config["device"]
    train_loader, test_loader, dataset_loader, num_train, num_test, num_dataset = get_data(config)
    config["num_train"] = num_train
    net = config["net"](bit).to(device)

    optimizer = config["optimizer"]["type"](net.parameters(), **(config["optimizer"]["optim_params"]))

    criterion = SFHCLoss(config, bit)

    Best_mAP = 0

    for epoch in range(config["epoch"]):

        current_time = time.strftime('%H:%M:%S', time.localtime(time.time()))

        print("%s[%2d/%2d][%s] bit:%d, dataset:%s, training...." % (
            config["info"], epoch + 1, config["epoch"], current_time, bit, config["dataset"]), end="")

        net.train()

        train_loss = 0
        for image, label, ind in train_loader:
            image = image.to(device)
            label = label.to(device)

            optimizer.zero_grad()
            u = net(image)

            loss = criterion(u, label.float(), ind, config)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

        train_loss = train_loss / len(train_loader)

        print("\b\b\b\b\b\b\b loss:%.3f" % (train_loss))

        if (epoch + 1) % config["test_map"] == 0:
            Best_mAP = validate(config, Best_mAP, test_loader, dataset_loader, net, bit, epoch, num_dataset)


if __name__ == "__main__":
    config = get_config()
    print(config)
    for bit in config["bit_list"]:
        config["pr_curve_path"] = f"log/alexnet/SFHC_{config['dataset']}_{bit}.json"
        train_val(config, bit)


{'alpha': 0.1, 'optimizer': {'type': <class 'torch.optim.rmsprop.RMSprop'>, 'optim_params': {'lr': 1e-05, 'weight_decay': 1e-05}}, 'info': '[DSH]', 'resize_size': 256, 'crop_size': 224, 'batch_size': 64, 'net': <class 'network_cifar.AlexNet'>, 'dataset': 'cifar10-1', 'epoch': 250, 'test_map': 15, 'save_path': 'save/DSH', 'device': device(type='cuda', index=1), 'bit_list': [48], 'topK': -1, 'n_class': 10, 'data': {'train_set': {'list_path': './data/cifar10-1/train.txt', 'batch_size': 64}, 'database': {'list_path': './data/cifar10-1/database.txt', 'batch_size': 64}, 'test': {'list_path': './data/cifar10-1/test.txt', 'batch_size': 64}}}
Files already downloaded and verified
train_dataset 5000
test_dataset 1000
database_dataset 59000


AssertionError: Torch not compiled with CUDA enabled